# Notebook for Lab 3 - Asymmetric Encryption
By Luis Daniel Casais Mezquida  
Cryptography 22/23  
Bachelor's degree in Applied Mathematics and Computing, grp. 121  
Universidad Carlos III de Madrid

## General goal
In this lab you’ll get familiarized with programming hash functions, MAC, key derivation functions,
authenticated encryption and signatures using pyca/cryptography library. 

## Specific goals
- Hash with SHA256 and SHA512
- MAC with HMAC and CMAC
- Key derivation functions with Scrypt and PBKDF2
- Authenticated encryption with Fernet and AES-GCM
- RSA-PSS signature scheme

## Modules
Cryptography library documentation can be found [here](https://pypi.org/project/cryptography). You’ll may find useful to have these links as reference:
- [Hash functions](https://cryptography.io/en/latest/hazmat/primitives/cryptographic-hashes/)
- [MAC](https://cryptography.io/en/latest/hazmat/primitives/mac/)
- [Key derivation functions](https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/)
- [Authenticated encryption with Fernet (Encrypt-then-MAC)](https://cryptography.io/en/latest/fernet/) (also [here](https://github.com/fernet/spec/blob/master/Spec.md))
- [RSA signatures](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/#signing)

## Lab assignment and assessment
You are given one Python script/notebook, that contains the headers of all the functions you should develop at the end of this lab. Next, you’ll be guided through several tasks you’ll have to complete in order to do so.

Lab 3 assessment will be performed with a test/quiz/exam at the end of the course (along the other lab assessments).

# 0. Modules installation and imports

In [2]:
import base64, os

from cryptography.hazmat.primitives import hashes, hmac, cmac
from cryptography.exceptions import InvalidSignature
from cryptography.fernet import InvalidToken
from cryptography.hazmat.primitives.ciphers import algorithms

from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.exceptions import InvalidKey, InvalidTag

from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
from cryptography.hazmat.primitives.asymmetric import utils

from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives import serialization

# 1. Message digests (Hash functions)
First, you’ll focus on cryptographic hash functions. A cryptographic hash function takes an arbitrary block of data and
calculates a fixed-size bit string (a digest), such that different data results (with a high probability) in different digests.

Read documentation [here](https://cryptography.io/en/latest/hazmat/primitives/cryptographic-hashes/).

Notice that every time you need to compute a digest you need to create a `hash` object (if you do not do this, you’ll get
an error).

### Task 1.1 Applying SHA256 hash function

Compute the hash of the given message using SHA256 algorithm. Check that you should obtain the following digest or hash value.

In [27]:
message = b'very important software that needs its integrity checked'

In [28]:
digest = hashes.Hash(hashes.SHA256())
digest.update(message)
digest_value_SHA256 = digest.finalize()

print(digest_value_SHA256)

b'\xb8\r\x9eN\xa8\xb6\x9dp\xbc\xb6\xd0\xcf\xb8\x90;\x99\xee\x96\xe0\xda\xa8\x0e5\xa1i\xfe\xbe\x9e\xb5\x9c\x1a\x8c'


The result should be:
```python
digest_value_SHA256 = b'\xb8\r\x9eN\xa8\xb6\x9dp\xbc\xb6\xd0\xcf\xb8\x90;\x99\xee\x96\xe0\xda\xa8\x0e5\xa1i\xfe\xbe\x9e\xb5\x9c\x1a\x8c'
b64_digest_value_SHA256 = b'uA2eTqi2nXC8ttDPuJA7me6W4NqoDjWhaf6+nrWcGow='
```

**Q:** Which is the length of the digest?  

**A:** 32B (256b)

In [16]:
print(len(digest_value_SHA256))

32


### Task 1.2 Applying SHA512 hash function
Repeat again the previous computation with the same message, but in this case, use SHA512 algorithm.

In [25]:
message = b'very important software that needs its integrity checked'

In [26]:
digest = hashes.Hash(hashes.SHA512())
digest.update(message)
digest_value_SHA512 = digest.finalize()

print(digest_value_SHA512)

b'=\xbf\xe6^\x8c\xf6\xfd\xae;\x96\xb8\xce\xeb\n\xe70\xe5=\x1d\xe2D%\xa4\xa6P\x81\xae\xdd"\xef\xb7 6\x99b\xc1\xb4\xf1_E\xce\x08\xd1\xdf\xacvG;\xfd\xb7\x8a\x89jDM?\x9ak\xd7\xbf\x0b\x99x\xfc'


The result should be:
```python
digest_value_SHA512 = b'=\xbf\xe6^\x8c\xf6\xfd\xae;\x96\xb8\xce\xeb\n\xe70\xe5=\x1d\xe2D%\xa4\xa6P\x81\xae\xdd"\xef\xb7 6\x99b\xc1\xb4\xf1_E\xce\x08\xd1\xdf\xacvG;\xfd\xb7\x8a\x89jDM?\x9ak\xd7\xbf\x0b\x99x\xfc'
b64_digest_value_SHA512 = b'Pb/mXoz2/a47lrjO6wrnMOU9HeJEJaSmUIGu3SLvtyA2mWLBtPFfRc4I0d+sdkc7/beKiWpETT+aa9e/C5l4/A=='
```

**Q:** Which is the length of the digest?  

**A:** 64B (512b)

In [14]:
print(len(digest_value_SHA512))

64


### Task 1.3 Comparing outputs with 1-bit different inputs
Check that by changing just one bit of the input, the digest is quite different. Use the following data:

In [23]:
data_to_hash_1 = b'abc123'
data_to_hash_2 = b'bbc123'

In [24]:
print("data 1:", data_to_hash_1.hex())
print("data 2:", data_to_hash_2.hex())

digest = hashes.Hash(hashes.SHA256())
digest.update(data_to_hash_1)
digest_value_1 = digest.finalize()

digest = hashes.Hash(hashes.SHA256())
digest.update(data_to_hash_2)
digest_value_2 = digest.finalize()

print("hash 1:", digest_value_1.hex())
print("hash 2:", digest_value_2.hex())

data 1: 616263313233
data 2: 626263313233
hash 1: 6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
hash 2: 8f983655cd9758784a83282091817945bea3fec41caa73068c23a2d68ccf382c


Notice that their binary representation only differs in one bit (you can easily check it by printing their hex value (<code>data_to_hash.hex()</code>). You can also use hexadecimal representation to assess the differences in the outputs.

# 2. Message Authentication Codes (MAC)

Now you’ll generate and verify Message Authentication Codes or MACs.  
Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/mac/).

## 2.1 HASH-BASED MESSAGE AUTHENTICATION CODE (HMAC)

Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/).

### Task 2.1.1 Computing an authentication tag with HMAC
Use the HMAC algorithm (combined with the hash function SHA256) to compute the authentication tag on the given message and using the given key.

Notice that if you want to use a fresh key you may use the following method provided by the library to get the size of
the digest (in the case of HMAC it would be wise to use a key of this size): 
<code>hmac_key = os.urandom(hashes.SHA256.digest_size)</code>

In [31]:
hmac_key = b'$\x7f0^\xcd\xce?5\xf3\x08\xaftq\xfc\x92"D5\xacQ\xbbz^P\xeej\x15\x83G\xbf\x86\xd2'
b64_hmac_key = b'JH8wXs3OPzXzCK90cfySIkQ1rFG7el5Q7moVg0e/htI='
message_to_mac = b'message to mac'

In [32]:
h = hmac.HMAC(hmac_key, hashes.SHA256())
h.update(message_to_mac)
hmac_auth_tag = h.finalize()

print(hmac_auth_tag)

b'k~\x91g=\x1aL\x05\xd10\x01\xaat$\xaf\x03@\x1dW\xc2\xc1\x9a\xe9m\xe2\xb6\r\xfce-\x91\xdc'


You should obtain the following authentication tag:
```python
hmac_auth_tag = b'k~\x91g=\x1aL\x05\xd10\x01\xaat$\xaf\x03@\x1dW\xc2\xc1\x9a\xe9m\xe2\xb6\r\xfce-\x91\xdc'
b64_hmac_auth_tag = b'a36RZz0aTAXRMAGqdCSvA0AdV8LBmult4rYN/GUtkdw='
```



### Task 2.1.2 Verifying an HMAC authentication tag
Now you’ll verify the authentication tag you have generated. We are going to capture the exception that may be raised when verification fails. You can use the following code, where we capture the exception <code>cryptography.exceptions.InvalidSignature</code>.

In [56]:
def verifyHMACsha256(hmac_key, message_to_mac, hmac_auth_tag):
    mac = hmac.HMAC(hmac_key, hashes.SHA256())
    try:
        mac.update(message_to_mac)
        mac.verify(hmac_auth_tag)
        print('The HMAC verification has been successful')
    except InvalidSignature:
        print('The HMAC verification has failed')

In [57]:
verifyHMACsha256(hmac_key, message_to_mac, hmac_auth_tag)

The HMAC verification has been successful


### Task 2.1.3 Failed HMAC verifications
Check that verification fails when you modify the message, the authentication tag or both.

In [58]:
verifyHMACsha256(hmac_key, message_to_mac + b"a", hmac_auth_tag)

bad_hmac_auth_tag = b'k~\x91g=\x1aL\x05\xd10\x01\xaat$\xaf\x03@\x1dW\xc2\xc1\x9a\xe9m\xe2\xb6\r\xfce-\x91\xd0'
verifyHMACsha256(hmac_key, message_to_mac, bad_hmac_auth_tag)

verifyHMACsha256(hmac_key, message_to_mac + b"a", bad_hmac_auth_tag)

The HMAC verification has failed
The HMAC verification has failed
The HMAC verification has failed


## 2.2 CIPHER-BASED MESSAGE AUTHENTICATION CODE (CMAC)

Now you’ll repeat the previous tasks but, in this case, you’ll use the CMAC algorithm.
Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/mac/cmac/).

In [42]:
cmac_key = b'\xf8&=\x1a$\xb4\xfa\x1a\xe2\xed\xe16.\xf9\x8af'
b64_cmac_key = b'+CY9GiS0+hri7eE2LvmKZg=='
message_to_mac = b'message to mac'

### Task 2.2.1 Computing an authentication tag with CMAC
Use the CMAC algorithm (combined with AES-128 block cipher) to compute the authentication tag on the
given message and using the given key.

Notice that if you want to use a fresh key you may use the following method provided by the library to get the size of the digest (in the case of HMAC it would be wise to use a key of this size) as shown next:

In [52]:
AES_128_BYTE_KEY_SIZE = algorithms.AES128.key_size
cmac_key = b'\x86\x16S\t\x80\xf1U\xd28\xb1%\x95\x8aMO\xb3'

In [53]:
c = cmac.CMAC(algorithms.AES128(cmac_key))
c.update(message_to_mac)
cmac_auth_tag = c.finalize()

print(cmac_auth_tag)

b'\xd9Va\xe4\xaa\x14\xa1\x1e:b `\x18\xa09E'


Check that the results are:
```python
cmac_auth_tag = b'\xd9Va\xe4\xaa\x14\xa1\x1e:b `\x18\xa09E'
b64_cmac_auth_tag = b'2VZh5KoUoR46YiBgGKA5RQ=='
```

### Task 2.2.2 Comparing HMAC and CMAC
Compare the length of the tag obtained with CMAC and with HMAC with the provided parameters and configurations.
Justify their values and provide reasons for their differences.

In [54]:
print("hmac length:", len(hmac_auth_tag), "\bB")
print("cmac length:", len(cmac_auth_tag), "\bB")

hmac length: 32B
cmac length: 16B


**A:** The length of the MAC is the length of the hash/algorithm key length used.

### Task 2.2.3 Verifying a CMAC authentication tag
Verify the authentication tag you have generated. You may have to capture the exception <code>cryptography.exceptions.InvalidSignature</code>.

In [61]:
def verifyCMACaes128(cmac_key, message_to_mac, cmac_auth_tag):
    mac = cmac.CMAC(algorithms.AES128(cmac_key))
    try:
        mac.update(message_to_mac)
        mac.verify(cmac_auth_tag)
        print('The CMAC verification has been successful')
    except InvalidSignature:
        print('The CMAC verification has failed')

verifyCMACaes128(cmac_key, message_to_mac, cmac_auth_tag)

The CMAC verification has been successful


### Task 2.2.4 Failed CMAC verifications
Check that verification fails when you modify the message, the authentication tag or both.

In [63]:
verifyCMACaes128(cmac_key, message_to_mac + b"a", cmac_auth_tag)

bad_cmac_auth_tag = b'\xd9Va\xe4\xaa\x14\xa1\x1e:b `\x18\xa090'
verifyCMACaes128(cmac_key, message_to_mac, bad_cmac_auth_tag)

verifyCMACaes128(cmac_key, message_to_mac + b"a", bad_cmac_auth_tag)

The CMAC verification has failed
The CMAC verification has failed
The CMAC verification has failed


# 3. Key Derivation Functions (KDF)

Key derivation functions derive bytes suitable for cryptographic operations from other data (like passwords) or
entropy-rich data sources.

Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/).

Notice that these functions may have two different goals:
- **Cryptographic key derivation**: In this case, we seek to enhance the quality of the key.
- **Password storage**: In this case, we seek to conceal the real value of the password but at the same time increase the computational effort an adversary should have to do in order to run a brute force attack.

## 3.1 Scrypt (Secure password storage)

You’ll first use Scrypt to compute the “hash” of a password (eg, you would store this value in a server’s database,
together with the needed additional parameters, to authenticate users by checking their passwords).

Read documentation [here](Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/#scrypt).

Notice that the salt should be randomly generated, but we provide it here so you can replicate the results.

### Task 3.1.1 Secure password storage with Scrypt
Compute the “hash” of the given password using the following parameters and configurations.

In [69]:
password = b'my great password'
salt = b'f?d\xbc\xf1\x94\xe2<\xad\xeb\x11\xa8\xb7q\xbc\xeb'
b64_salt = b'Zj9kvPGU4jyt6xGot3G86w=='
length = 32
n = 2**14
r = 8
p = 1

In [70]:
kdf = Scrypt(salt, length, n, r, p)
hashed_pwd = kdf.derive(password)

print(hashed_pwd)

b'\x0e\xc6M\xf7In\x97\xce\x87\x8c\xce\x06\xceo\xe2\xeb\x18\x02*v\xa2=c\xfd\x14\xba\xe0\xf7 \xc2\xf0\x97'


Check that the results are:
```python
hashed_pwd = b'\x0e\xc6M\xf7In\x97\xce\x87\x8c\xce\x06\xceo\xe2\xeb\x18\x02*v\xa2=c\xfd\x14\xba\xe0\xf7 \xc2\xf0\x97' 
b64_hashed_pwd = b'DsZN90lul86HjM4Gzm/i6xgCKnaiPWP9FLrg9yDC8Jc='
```

### Task 3.1.2 Password verification with Scrypt
Now verify the password against the “hash” of the password using the same parameters and configuration.

In [75]:
kdf = Scrypt(salt, length, n, r, p)
kdf.verify(password, hashed_pwd)

### Task 3.1.3 Failed password verification with Scrypt
Try to verify the “hashed” password against another (incorrect) password. Notice that you have to capture the
exception <code>cryptography.exceptions.InvalidKey</code>.

In [77]:
try:
    kdf = Scrypt(salt = salt, length = 32, n = 2**14, r = 8, p = 1)
    kdf.verify(password + b"0", hashed_pwd)
    print("Verification passed")
except InvalidKey:
    print("Verification failed")

Verification failed


## 3.2 PBKDF2 (Password Based Key Derivation Function 2)

Now, you’ll use PBKDF2 (combined with a SHA256 based HMAC) to derive a high-quality key from a password.

Read documentation [here](https://cryptography.io/en/latest/hazmat/primitives/key-derivationfunctions/#cryptography.hazmat.primitives.kdf.pbkdf2.PBKDF2HMAC).

### Task 3.2.1 Generating high-quality keys from a password with PBKDF2
Compute the derived key from the given password using the following parameters and configurations.

_Note: We will not practice the verification of PBKDF2 as it is not necessary if used for generating higher quality keys. However, it can be verified similarly to Scrypt function._

In [79]:
pwd = b'my great password'
salt = b'\x00\x10\xe1e\xbek\xeeSGd\xb5(=\x1f\xaft'
b64_salt = b'ABDhZb5r7lNHZLUoPR+vdA=='
algorithm = hashes.SHA256()
length = 32
iterations = 100000

In [81]:
kdf = PBKDF2HMAC(algorithm, length, salt, iterations)
derived_key = kdf.derive(pwd)

print(derived_key)

b'\xbeW\x06U\xc2\xb8\xf8\xdai\xf5\x12k\x18\xe8\x82\xdb\xde/\xfcnB\xf7\x1b\xa6\xe1J\xa1\x82\x984+\xe2'


Check the results:
```python
derived_key = b'\xbeW\x06U\xc2\xb8\xf8\xdai\xf5\x12k\x18\xe8\x82\xdb\xde/\xfcnB\xf7\x1b\xa6\xe1J\xa1\x82\x984+\xe2'
b64_derived_key = b'vlcGVcK4+Npp9RJrGOiC294v/G5C9xum4Uqhgpg0K+I='
```

# 4. Authenticated Encryption

## 4.1 FERNET

Fernet is a system for symmetric encryption/decryption, using current best practices. It also authenticates the
message, which means that the recipient can tell if the message has been altered in any way from what was originally
sent. Fernet is included in the cryptography library. It is implemented with AES 128 in CBC mode and SHA256
HMAC.

To encrypt and decrypt data, we will need a secret key that must be shared between anyone who needs to encrypt or
decrypt data. It must be kept secret from anyone else, because anyone who knows the key can read and create
encrypted messages. This means that we will need a secure mechanism to share the key. The same key can be used
multiple times.

Read documentation [here](https://cryptography.io/en/latest/fernet/) (and also [here](https://github.com/fernet/spec/blob/master/Spec.md)).

### Task 4.1.1 Encrypting a message with Fernet

To encrypt (and authenticate) a message, we must first create a Fernet object using a previously created key. We then
call the encrypt function, passing the data we wish to encrypt in the form of a byte array.

Encrypt the given message using Fernet with the given key given.

Notice that although in this lab assignment the key is provided to you, you should randomly generate new ones for
each session.

Notice also that the key and the token are coded following a variant of base64 which is safe for URL encoding:
<code>base64URL</code>.

In [83]:
message = b'my deep dark secret'
fernet_key = b'jdOQiny3SqoBvqNyFUSrGSG_rYzoDoPIONR21LTRCxk='  # fernet_key = Fernet.generate_key()

In [88]:
f = Fernet(fernet_key)
fernet_token = f.encrypt(message)

print(fernet_token)

b'gAAAAABjySvt0Z7pM5EwHoNc_SdBe5jjKbZUe6Ql4jg6EllgQc1sWkHzRfbUZN3QsUQ0JglIdN-R1XnXjwaEawoikYSqL3irLihCYQVi6hg2yyP-mI4az7s='


You should get something *similar* to the token shown next (notice that it should be different, as the IV, is with a high probability different). Notice that this token contains the HMAC authentication tag, several parameters (version, timestamp), the IV for CBC and the AES ciphertext:
```python
fernet_token = b'gAAAAABfs9hj_1geoztj4Toon--b6IKAHZtUhYAofYN5jK29l-9_1i-Gb9R6AOQqZSpTSyX8-0fIgbY8fgNUz7gmVhmAYiQ1qJS1xp0eMaZHOWrQlBTTgfA='
```

### Task 4.1.2 Decrypting a message encrypted with Fernet

To decrypt (and verify) a Fernet token, you must again create a Fernet object using the same key that was used to
encrypt (and authenticate) the data. You then have to call the decrypt function, passing to it the Fernet token in the
form of a byte array. The function returns the decrypted original message if everything is correct.

Verify the Fernet token you obtained in the previous task.

In [85]:
print(f.decrypt(fernet_token))

b'my deep dark secret'


### Task 4.1.3 Failed Fernet token verification

Modify the token and check that the decryption (and verification) raises an error. It could be a good idea to capture the exception <code>cryptography.fernet.InvalidToken</code>. To make the modifications at the beginning of the Fernet token, you can use, for example, <code>corrupted_token = b'01a2' + fernet_token </code>.

In [90]:
corrupted_token = b'01a2' + fernet_token
try:
    pt = f.decrypt(corrupted_token)
    print("Verification successful")
except InvalidToken:
    print("Verification failed")

Verification failed


## 4.2 AES-GCM

Another way to implement both encryption and authentication is using an authenticated encryption operation mode.
These modern operation modes encrypt the plaintext and also generate an authentication tag, in both cases using
symmetric cryptography. Encryption is done with a symmetric cipher and authentication with a MAC.

In this Lab you’ll use one of these special operation modes, particularly, the Galois Counter Mode (GCM).
GCM implements what is known as authenticated encryption with additional data (AEAD). In this case data used as
input to the algorithm is composed by two different parts:
- plaintext: this will be the part of the input that will be encrypted and authenticated
- associated data: this part of the input will NOT be encrypted, but will be authenticated

Read the documentation of the function that already integrates AES with GCM [here](https://cryptography.io/en/latest/hazmat/primitives/aead/#cryptography.hazmat.primitives.ciphers.aead.AESGCM).

### Task 4.2.1 Encrypting (and authenticating) with AES-GCM

Encrypt with AESGCM the following plaintext (data) and associated data (aad) using the parameters given in Code
Snippet 12.

Notice that although in this lab assignment the key is provided to you, you should randomly generate new ones each
time you would like to use AESGCM.

Notice also that the documentation of this class repeats multiple times that YOU SHOULD NEVER REUSE A NONCE
with the same key. Doing so makes the encryption completely vulnerable.

In [91]:
data = b"my super mega secret message"
aad = b"authenticated but unencrypted data"
aesgcm_key = b'\x00A \x07a\x1c\xa4T\xd8N\xa6\x1c\xae\x89\xd2\xde'
b64_aesgcm_key = b'AEEgB2EcpFTYTqYcronS3g=='
aesgcm_nonce = b'&\xa8\xc4\xb0\xf6\xbf\xd4\xea\xe3UU+'
b64_aesgcm_nonce = b'JqjEsPa/1OrjVVUr'

In [92]:
aesgcm = AESGCM(aesgcm_key)
ciphertext_tag = aesgcm.encrypt(aesgcm_nonce, data, aad)

print(ciphertext_tag)

b'\xea\x80\x94\t(\x9e\xda\xcdo\xdc\xb7\x048\x8e\xb6\xaa6\nZu\xa7y\xe0n\xa8\x0bv\xcd\xf3\xd6\xf9\xec\x85j\xcco\xa3\xc2\xa1\xd2n\xb5|\x18'


Check the results are:
```python
ciphertext_tag = b'\xea\x80\x94\t(\x9e\xda\xcdo\xdc\xb7\x048\x8e\xb6\xaa6\nZu\xa7y\xe0n\xa8\x0bv\xcd\xf3\xd6\xf9\xec\x85j\xcco\xa3\xc2\xa1\xd2n\xb5|\x18'
b64_ciphertext_tag = b'6oCUCSie2s1v3LcEOI62qjYKWnWneeBuqAt2zfPW+eyFasxvo8Kh0m61fBg='
```

### Task 4.2.2 Decrypting (and verifying) an AES-GCM ciphertext (and authentication tag)
Verify your ciphertext and tag. You should get the original plaintext.

In [93]:
print(aesgcm.decrypt(aesgcm_nonce, ciphertext_tag, aad))

b'my super mega secret message'


### Task 4.2.3 Failed AES-GCM decryption (and authentication)
Modify the ciphertext and check that the decryption and verification process fail. You may have to capture the
exception <code>cryptography.exceptions.InvalidTag</code>.

In [95]:
corrupted_tag = b'01a2' + ciphertext_tag
try:
    pt = aesgcm.decrypt(aesgcm_nonce, corrupted_tag, aad)
    print("Verification successful")
except InvalidTag:
    print("Verification failed")

Verification failed


# 5. RSA Signatures

## 5.1 RSA SIGNATURE GENERATION (SIGNING)

A digital signature is a byte-like object that allows verifying the authenticity of digital messages or documents. Digital
signatures schemes use asymmetric cryptography. In this Lab you’ll use RSA signature generation and verification
functions.

Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/#signing).

### Task 5.1.1 Reading the contents of a file
Create a file <code>files/payload.dat</code> within the Lab environment. Write something (ascii) in the file. Save it.

Now you can read its contents (as binary data) with the following code:

In [108]:
with open('files/payload.dat', 'rb') as f:
    payload = f.read()

Retrieve the content of the file (by reading it) and print it with Python.

In [109]:
with open('files/payload.dat', 'rb') as f:
    payload = f.read()

print(payload)

b'en el aeropuerto hay avione'


### Task 5.1.2 Signing with RSA-PSS

To perform the signature, you may use one of the asymmetric keys generated previously in Lab2 or generate a
new keypair. Which one do you have to use to generate the signature, the public or the private one? Check the
library documentation to answer this question or review lecture materials!

Sign the contents of the file <code>files/payload.txt</code>. Use PSS padding and SHA256 hash. For PSS padding, use the parameters given next.

In [100]:
mgf = asym_padding.MGF1(hashes.SHA256())
salt_length = asym_padding.PSS.MAX_LENGTH

In [105]:
# load the keys
PASSWORD = b"pene"

with open("files/priv_key.pem", 'rb') as pem_in: 
    priv_key = load_pem_private_key(pem_in.read(), PASSWORD)
    
with open("files/pub_key.pem", 'rb') as pem_in: 
    pub_key = load_pem_public_key(pem_in.read())

# signing
signature = priv_key.sign(payload, asym_padding.PSS(mgf, salt_length), hashes.SHA256())

print(signature, type(signature))


b'\x98\x1e\xee\x106?\x17qL2"\x8f\xf2%&\xa1\r\xc9=\x9f1\xd7\x0bM\xf9\x1b\x8fvr\x11\x119\xef\xa8\x03t\xf9/\x00\xc8\xafJpi\xcf)Es\xb3 \x91\xfd|\x1f\xc8Hb\xc3\x1e\xb5`b\xa9\xe1\xcd4e\x02*\xb2\x93\x81\xd1\x93\xae\xa1,rT\x01\x06\xe7\x14\xb2U\xb5\'8wp\xcd2m:\xee\xd9\x951\xd6\x196{\x83\xe1\xed\xe9\xc7\xd6b\xbc$\xc8\xb8\x90\\e\xea\n\xcf\x18aq\xff\xbfj\x19(m\xcf\xe1=!kz\xc9Y\xf9o\xcfi\xa9\x86H\xe7T\xc1\x05\x87.\x1d&r\xa7\xe9\xb99\xb1J\x06\x07\xf2=w\xee\xff\xb9\xabiG\x11e \xb4\xf4ix\xd8\xc7\xfeUTY\x0e\xd1\xd8#l\x89\xde\xff$\xe80\xe8\xf3\xbc\x08=]4G\xed\xd9`\xff\x04h\x11#>\xeb\xed\xaa}r\xc9\xbd\xd2\xdb\xfa\xeba\xbb\x99\xc7*\xaa\xe6\xef\x08\x1d\x1c[\x06\xf3S\xad%:\x935s\x82\xe98\xbb\xdc\xc1\xe8\xf8\xc9\xa5\xad\xbeH\x85' <class 'bytes'>


### Task 5.1.3 Saving data to a file
After generating the signature, save it in <code>files/signature.sig</code>.

In [103]:
with open('files/signature.sig', 'wb') as f:
    f.write(signature)

Have a look at the contents of this file using the IDE you are using (or a simple text editor like NotePad++ or similar).
If you do not like what you see, encode the data before saving with base64 encoding or in hexadecimal.

In [114]:
with open('files/signature_hex.sig', 'w') as f:
    f.write(signature.hex())

## 5.2 RSA SIGNATURE VERIFICATION

To verify the document signed in the previous section it is necessary that you use the corresponding key, and the
contents of the files <code>payload.dat</code> and <code>signature.sig</code>. If you encoded the signature, remember to decode it before the verification.

Read the documentation [here](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/#verification).

### Task 5.2.1 Verifying an RSA-PSS signature

First, make sure you know the key you should use, the public or the private one?

Write the code needed to verify the previous signature using the same algorithm parameters. Note that it may raise
the exception <code>cryptography.exceptions.InvalidSignature</code> if the signature isn’t valid.

In [115]:
# read files
with open('files/payload.dat', 'rb') as f:
    payload = f.read()
with open('files/signature.sig', 'rb') as f:
    signature = f.read()

# verify (w/ public key)

pub_key.verify(signature, payload, asym_padding.PSS(mgf, salt_length), hashes.SHA256())

### Task 5.2.2 Failed RSA-PSS signature verification

Sign the document again and before verifying it:
- Check that the generated signature is different
- Change some character within the contents of <code>payload.txt</code> and check that the verification fails.

In [117]:
corrupted_payload = payload + b"0"

try:
    pub_key.verify(corrupted_payload, payload, asym_padding.PSS(mgf, salt_length), hashes.SHA256())
    print("Verification successful")
except InvalidSignature:
    print("Verification failed")

Verification failed
